# Final Notebook-Comprehensive Notebook detailing the process to the final model and project overview.

# Overview of Project
 This project aims to utilize Illinois 2018 American Community Survey data to create an imcome predicitor to aid Illinois and governmental programs with their allocation of the programs. In particular programs such as SNAP and unemployment. This will allow for a faster application process due to better accuracy. This is very crucial in especially when looking at unemployment. This has been one of the most vital neccesities of this COVID-19 age due to the massive amount of layoff and furloughs done by organizations. In creating a better system to access these individuals and their income we can better deploy these services to the citizens that need them. More information on this topic can be found [here]
 
# Data acquisition 
 Prior to running any of the Jupyter Notbooks or data in this download the neccesary files andenvironments..

Download [Andaconda](https://docs.anaconda.com/anaconda/install/) to be able to install the environment and use your preference of commandline options or Editor to view the data.
Also go to this [Census Link](https://www2.census.gov/programs-surveys/acs/data/pums/2018/5-Year/) and down load the csv_hil.zip to acquires the data used in this project.

### From the list options below choose the environments within this neccesary envrionment for your operating system.

[`tens.yml`]

To utilize Geopandas:

[`Capstone.yml`]

All files are located in the main directory and src folder.
Run these two lines of code after the download to apply the kernals to your IDE.

`python -m ipykernel install --user --name tens --display-name "Python 3 (tens)"` 

`python -m ipykernel install --user --name Capstone --display-name "Python 3 (Capstone)"`  

 
# Goals:
This project aims to:
- Build a modle that predicts an individuals income based on multiple parameters such as property value, proterty taxes and number of people in a household.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os 
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
# from src.js_functions import coeff_determination
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from tensorflow.keras.callbacks import EarlyStopping 
from src.js_functions import  process_data
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsRegressor
import tensorflow as tf
from keras.layers import Dense
from keras.models import Sequential
import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


**Importing Data**

In [2]:
df=pd.read_csv("../../Data/csv_hil/psam_h17.csv")

**The orginal dataframe will now processed via the process data function. For more about the cleaning and processing of the data [check out this notbook](../Explortatory/Data_cleaning_processing.ipynb) as well as [this py file](../../src/js_functions.py).**


In [3]:
X_train, X_test,y_train, y_test=process_data(df)

**Looking at the first 5 rows of cleaned dataset**

In [4]:
X_train.head()

,SMOCP,NP,BDSP,INSP,RMSP,VALP,TAXAMT,ACR_<1.0>,ACR_<2.0>,ACR_<3.0>,...,YBL_<13.0>,YBL_<14.0>,YBL_<15.0>,YBL_<16.0>,YBL_<17.0>,YBL_<18.0>,YBL_<19.0>,YBL_<20.0>,YBL_<21.0>,YBL_<22.0>
6894,652.0,3,3.0,600.0,5.0,130000.0,3850.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49998,1206.0,4,3.0,400.0,4.0,125000.0,3150.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
182231,1042.0,1,1.0,890.0,5.0,165000.0,3650.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
80188,5217.0,4,4.0,1200.0,7.0,650000.0,14000.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
292305,2188.0,2,3.0,2300.0,9.0,800000.0,14500.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Data Understanding
Each row represents a household.
Below are the features for a given household.

`y_train and y_test ` represent household income

**SMOCP** -Selected monthly owner costs.

**NP** -Number of persons associated with this housing record.

**BDSP** -Number of bedrooms.

**INSP** -5Fire/hazard/flood insurance yearly amount.

**RMSP**-Number of rooms. 

**VALP** -Property value. 

**TAXAMT** -Property taxes. 

**ACR** -Lot size
   - Each column for this feature represent differnt house lot sizes in a range

**FS** -Yearly food stamp/Supplemental Nutrition Assistance Program (SNAP)
recipiency

   - Each column for this feature tell us where some one does or does not have SNAP

**TOIL**-Flush toilet
   - Each column for this feature has a toliet that flushes or not

**VEH** -Number of vehicles
   - Each column for this feature represents a differnt numbers of cars

**YBL**-When structure first built
   - Each column for this feature represent differnt house lot sizes in a range

# Modeling

In [5]:
#This is our baseline model.
lr=LinearRegression()

In [6]:
lr.fit(X_train,y_train)

LinearRegression()

In [7]:
lr.score(X_train,y_train)

0.3847594087743529

In [8]:
scores= cross_val_score(lr,X_train,y_train,cv=5)

In [9]:
scores.mean()

0.38422207389115826

In [ ]:
adj_rsqaured(lr,X_train,y_train)

**Basline model has a R-squared score of .38 so not to bad as basline model. Let's see if we can increase this value through a different algorithm. Overall, This initial model is telling us that 38% of the variance in the target column can be explained by the features we are using**

In [10]:
knr=KNeighborsRegressor(n_neighbors=5)
knr.fit(X_train,y_train)
knr.score(X_train,y_train)

0.4883788705468357

In [11]:
scores1= cross_val_score(knr,X_train,y_train,cv=5)

In [12]:
scores1.mean()

0.25017895465649836

In [ ]:
adj_rsqaured(knr,X_train,y_train)

**The with the KNN model the features account for 25% of the variance seen in the target column.**

**Lets run a Random forest regressor and see how this may preform.

In [13]:
rfr=RandomForestRegressor(max_depth=10, min_samples_split= 4, n_estimators = 100)

In [14]:
rfr.fit(X_train,y_train)

RandomForestRegressor(max_depth=10, min_samples_split=4)

In [15]:
rfr.score(X_train,y_train)

0.4711950787125593

In [16]:
scores2= cross_val_score(rfr,X_train,y_train,cv=5)

In [17]:
scores2.mean()

0.40715365539405746

In [ ]:
adj_rsqaured(rfr,X_train,y_train)

In [18]:
# The R-squared predictor
from keras import backend as K
def coeff_determination(y_true, y_pred):
    """https://jmlb.github.io/ml/2017/03/20/CoeffDetermination_CustomMetric4Keras/"""

    SS_res =  K.sum(K.square( y_true-y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

early_stop = EarlyStopping(monitor='val_loss', min_delta=1e-08, patience=0, verbose=1,
                           mode='auto')
callbacks_list = [early_stop]
model=Sequential()
model.add(Dense(50 ,activation="relu",input_dim=47))
model.add(Dense(25 ,activation="selu"))
model.add(Dense(1 ,activation='linear'))
model.compile(loss="mse",optimizer="adam",metrics=[tf.keras.metrics.RootMeanSquaredError(),"mae",coeff_determination])

In [19]:
history=model.fit(X_train,y_train,epochs=50,batch_size=50,validation_split=.2,callbacks=callbacks_list)

Train on 157760 samples, validate on 39441 samples
Epoch 1/50
157760/157760 [==============================] - 6s 38us/step - loss: 6525606460.7789 - root_mean_squared_error: 84935.6641 - mae: 49279.6484 - coeff_determination: 0.1464 - val_loss: 6560731825.3489 - val_root_mean_squared_error: 80860.3359 - val_mae: 47463.8516 - val_coeff_determination: 0.2373
Epoch 2/50
157760/157760 [==============================] - 6s 39us/step - loss: 6046045758.8763 - root_mean_squared_error: 80055.3594 - mae: 48082.0469 - coeff_determination: 0.2152 - val_loss: 6178551325.9871 - val_root_mean_squared_error: 79446.1875 - val_mae: 48200.5625 - val_coeff_determination: 0.2413
Epoch 3/50
157760/157760 [==============================] - 6s 37us/step - loss: 5989999164.2515 - root_mean_squared_error: 79081.4688 - mae: 47761.4453 - coeff_determination: 0.2244 - val_loss: 6348603322.9973 - val_root_mean_squared_error: 78870.1172 - val_mae: 46758.9570 - val_coeff_determination: 0.2585
Epoch 00003: early sto

**Seeing as the Randrom forest Regressor performed the best let's test it on our test set.**

In [20]:
rfr.score(X_test,y_test)

0.41607874904279185

In [ ]:
adj_rsqaured(rfr,X_test,y_test)

 **The model performs well on the test data where the level of vairance association is not to far off from the training data. With some more iteration we may be able to increase this value and create are more accurate model**

In [21]:
y_hat=rfr.predict(X_test)

In [22]:
print("Actual=%s, Predictied=%s" % (y_test.values.mean(),y_hat.mean()))

Actual=86708.29873633395, Predictied=86690.58098378546


**From this we can see that our actual value  and predicted value average do not differ to much which makes sense due to the model trying to acomodate all the data**

In [23]:
print("Actual=%s, Predictied=%s" % (y_test.values[0],y_hat[0]))

Actual=35000.0, Predictied=64016.95833267796


**However, when we look at the individual values and their predicted income counterparts there is a large discrepancy in some case such as the first value of the test data being 35,000 and the predcited value being 64,000. This is something I expected due to combining all of the counties in one dataframe. This is due to the cost of living and overall socio-economic advantages or disadvantages of a county based on it geographical location**

### End analysis conlusion:
- The final model predicts about 40% of the vairiance seen within the data. 
- When you look at individual values there is about a 29,000 dollar differnce in those values.

### Next steps:
- I would like to continue to Iterate on the current model and reduce the amount of vairance seen between the actual and predicted values.
- I would also like to subset the counties and see how that would affect the predicted values due to haveing more geographically related data.
- Moreover, I would like to do some more feature enginerring to possiblty increase the R-Squared value.
- Lastly, I would like to find more recent census data to have a better estimate to the current day household income.